#**Genetic CNN-(76)**

####Ayush Agrawal - 2018A8PS0469P
####Aryan Gupta  -  2018A7PS0017P
####Saurav Biyani - 2018A3PS0461P

In [ ]:
!pip install deap
!pip install torchsummary

     |████████████████████████████████| 163kB 8.8MB/s 


##**Importing Libraries**

In [ ]:
import random 
import numpy as np 
from deap import base, creator,tools,algorithms
from scipy.stats import bernoulli
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
from torchsummary import summary


##**CNN IMPLIMENTED**



Class Net takes the chromosome as input and makes the corresponding network.This network is trained to get the fitness function.

In [ ]:
class Net(nn.Module):
    def __init__(self,chromosome):
        super(Net, self).__init__()
        self.chromosome=chromosome
        
        ## first conv layer c5@20
        self.conv1 = torch.nn.Conv2d(in_channels= 1 , out_channels=20,kernel_size=5,stride = 1,padding= 2)

        ## for network 1

        self.conv2 = torch.nn.Conv2d(in_channels=20,out_channels=20,kernel_size=3,stride=1,padding=1)
        self.conv3 = torch.nn.Conv2d(in_channels=20,out_channels=20,kernel_size=3,stride=1,padding=1)
        self.conv4 = torch.nn.Conv2d(in_channels=20,out_channels=20,kernel_size=3,stride=1,padding=1)

        ## max pool1 mp2s2

        self.pool1 =torch.nn.MaxPool2d(kernel_size=2,stride=2)

        ## second conv layer c5@50
        self.conv5 = torch.nn.Conv2d(in_channels=20,out_channels=50,kernel_size=5,stride=1,padding = 2)

        ## for network2
        self.conv6 = torch.nn.Conv2d(in_channels=50,out_channels=50,kernel_size=3,stride=1,padding = 1)
        self.conv7 = torch.nn.Conv2d(in_channels=50,out_channels=50,kernel_size=3,stride=1,padding = 1)
        self.conv8 = torch.nn.Conv2d(in_channels=50,out_channels=50,kernel_size=3,stride=1,padding = 1)
        self.conv9 = torch.nn.Conv2d(in_channels=50,out_channels=50,kernel_size=3,stride=1,padding = 1)
        self.conv10 = torch.nn.Conv2d(in_channels=50,out_channels=50,kernel_size=3,stride=1,padding = 1)

        ## maxpooling layer 2 mp2s2
        self.pool2 = torch.nn.MaxPool2d(kernel_size=2,stride=2)

        ## lesnet layer
        self.fc1 = nn.Linear(7*7*50,500)
        self.fc2 = nn.Linear(500,10)

        ############################## initialization finished ############################3


   # forward method does one feedforward of the whole network and returns the output as predicted class.
    def forward(self, x):
      gene = self.chromosome
      ## out1 output of first conv layer
      out1 = F.relu(self.conv1(x))
           
      ## out2 output of network1  
        
      #first node
      out_1 = F.relu(self.conv2(out1))
        
      #second node
      if gene[0]==1:
        out_2 = F.relu(self.conv3(out1+out_1))
      else:
        out_2 = F.relu(self.conv3(out_1))

      #third node
      y = out1.clone()
      if gene[1]==1:
         y += out_1 
      if gene[2]==1:
         y += out_2 
      out2 = F.relu(self.conv4(y))
      
      ## out3 output maxpool
      out3 = self.pool1(out2)
     
      ## out4 output conv layer 2
      out4 = F.relu(self.conv5(out3))
       
      ## out5 output of network2 
      
      # first node
      out_1_1 = F.relu(self.conv6(out4))

      # second node
      x1 = out4.clone()
      if gene[3]==1:
        x1+= out_1_1
      out_2_1 = F.relu(self.conv7(x1))

      # third node
      x2 = out4.clone()
      if gene[4]==1:
        x2+=out_1_1
      if gene[5]==1:
        x2+= out_2_1
      out_3_1 = F.relu(self.conv8(x2))

      # fourth node
      x3 = out4.clone()
      if gene[6]:
        x3+=out_1_1
      if gene[7]:
        x3+=out_2_1
      if gene[8]:
        x3+=out_3_1
      out_4_1 = F.relu(self.conv9(x3))

      # fifth node
      x4 = out4.clone()
      if gene[9]:
        x4+= out_1_1
      if gene[10]:
        x4+=out_2_1
      if gene[11]:
        x4+=out_3_1
      if gene[12]:
        x4+=out_4_1
      out5 = F.relu(self.conv10(x4))

      ## out 6 output of max pool 2
      out6 = self.pool2(out5)
      
      ##flattening out6 to feed to linear layer
      out7 = torch.Tensor.reshape(out6,(1,2450))

      ## out7 is output of first lenar layer
      out8 = F.relu(self.fc1(out7))
      
      ## out 8 is output of last layer after applying softmax
      out9 = F.softmax(self.fc2(out8))
      
      return out9
        



##**Training function**

Takes input as network, training dataset, corresponding chromosome etc. and trains the network on the training dataset.

In [ ]:
def trainFunction(net,trainData,chromosome,optimizer,criterion):
    for epoch in range(totalEpochs):
        currentLoss=0.0
        for i,data in enumerate(trainData,0):

            inputs,labels=data
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()
            
            outputs=net(inputs)
            loss=criterion(outputs,labels)
            loss.backward()
            optimizer.step()
            currentLoss +=loss.item()
            if(i%20000)==19999:
                print("[%d,%5d] loss: %.3f" %(epoch+1,i+1,currentLoss/20000))
                currentLoss=0.0
                
    print("Finished Training")
    return net


##**Testing Function**

Test function takes the network and Test Dataset and returns the Testing accuracy.

In [ ]:
def testFunction(net,testLoader):
    correct=0
    total=0
    with torch.no_grad():
        for data in testLoader:
            images,labels=data
            images, labels = images.to(device), labels.to(device)
            
            outputs=net(images)
            _,predicted=torch.max(outputs.data,1)
            total+=labels.size(0)
            correct+=(predicted==labels).sum().item()
    accuracy=correct/total
    print("Accuracy: "+str(accuracy))
    return np.float32(accuracy),

##**Score Function**

Score function takes chromosome as input and returns the fitness value of the chromosome by training and then testing.

In [ ]:
def scoreFunction(chromosome):
  print(chromosome)
  net = Net(chromosome)
  
  net.to(device)
  print(net)
  criterion = nn.CrossEntropyLoss()
  optimizer = optim.Adam(net.parameters(), lr=0.00001)
  net=trainFunction(net,trainloaderMnist,chromosome,optimizer,criterion)
  return testFunction(net,testloaderMnist)

##**Loading Training and Testing Dataset**

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor()])

trainsetMnist=torchvision.datasets.MNIST(root='./data', train=True,download=True, transform=transform)
trainloaderMnist = torch.utils.data.DataLoader(trainsetMnist, batch_size=1,
                                          shuffle=True, num_workers=1)

testsetMnist = torchvision.datasets.MNIST(root='./data', train=False,download=True, transform=transform)
testloaderMnist = torch.utils.data.DataLoader(testsetMnist, batch_size=1,
                                         shuffle=False, num_workers=1)
classes = ('0', '1', '2', '3',
           '4', '5', '6', '7', '8', '9')

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw




Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


##**Main Model**

Model function runs the genetic algorithm for input number of generations for input population size and find the best structure for image recognition CNN.

In [ ]:
def Model(num_Epochs = 5, popl_size = 5, num_gen = 5, crossover_pb = 0.4, mutation_pb = 0.05):
    global totalEpochs
    totalEpochs = num_Epochs
    global chromosomeSize
    chromosomeSize = 13
    global population_size
    population_size = popl_size
    global num_generations
    num_generations = num_gen

    

    creator.create("FitnessMax", base.Fitness, weights = (1.0,))
    creator.create("Individual", list , fitness = creator.FitnessMax)

    toolbox = base.Toolbox()
    toolbox.register("binary", bernoulli.rvs, 0.5)
    toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.binary, n = chromosomeSize)
    toolbox.register("population", tools.initRepeat, list , toolbox.individual)

    toolbox.register("mate", tools.cxOrdered)
    toolbox.register("mutate", tools.mutShuffleIndexes, indpb = 0.5)
    toolbox.register("select", tools.selRoulette)
    toolbox.register("evaluate", scoreFunction)

    popl = toolbox.population(n = population_size)
    result = algorithms.eaSimple(popl, toolbox, cxpb = crossover_pb, mutpb = mutation_pb, ngen = num_generations, verbose = True)

    best_individuals = tools.selBest(popl, k = 3)
    for bi in best_individuals:
        print(bi)
    return best_individuals


#**END OF CODE**